This is a test, can I push to github now?

In [2]:

# Optional demo on a small subset to avoid O(n^2) blowup
subset = min(400, distance_matrix.shape[0])  # adjust as needed
if subset >= 3: # ensures there are at least 3 objects
    Z = linkage(squareform(distance_matrix[:subset, :subset], checks=False), method='average')
    labels = fcluster(Z, t=0.7, criterion='distance')
    print("Cluster labels (first few):", labels[:10])
else:
    print("Not enough items for clustering demo.")

Cluster labels (first few): [ 98  72  48  42  35  70  31 155  13 205]
